# Lab_2

## Numeric solution

In [36]:
import math
import random

import sys
from fractions import Fraction



## Functions

In [41]:
def get_rand_max_index(arr):
    max_i = []
    max_el = max(arr)
    i = arr.index(max_el)
    max_i.append(i)
    while i < len(arr):
        try:
            i = arr.index(max_el, i + 1)
            max_i.append(i)
        except ValueError:
            break
    return random.choice(max_i)

def get_rand_min_index(arr):
    min_i = []
    min_el = min(arr)
    i = arr.index(min_el)
    min_i.append(i)
    for i in range (min_i[0] + 1, len(arr)):
        if arr[i] == min_el:
            min_i.append(i)
    return random.choice(min_i)

def get_row_by_index(matrix, index):
    return matrix[index]

def get_column_by_index(matrix, index):
    return [matrix[i][index] for i in range(len(matrix))]

def get_max_index(arr):
    return arr.index(max(arr))

def get_min_index(arr):
    return arr.index(min(arr))

def vector_addition(a, b):
    return [i + j for i, j in zip(a, b)]

def printHeader(curr_file):
    if curr_file:
        print(header_formatter.format("k", "A strat", "B strat", "Win A", "Loss B", "UpBound", "LowBound", "Eps"), file=curr_file)
        print(110 * "-", file=curr_file)

def printLine(curr_file, k, A, B, win_a, loss_b, upper_bound, lower_bound, eps):
    if curr_file:
        print(line_formatter.format(k, A, B, win_a[0], win_a[1], win_a[2], loss_b[0], loss_b[1], loss_b[2], upper_bound.numerator, upper_bound.denominator, lower_bound.numerator, lower_bound.denominator, eps.numerator, eps.denominator), file=curr_file)

def brown_robinson_method(C, eps):
    m = len(C)    # A player strategies: strategy row consists of win of A
    n = len(C[0]) # B player strategies: strategy column consist of loss of B

    x = m * [0]
    y = n * [0]

    curr_strategy_a = 0
    curr_strategy_b = 0

    win_a = m * [0]
    loss_b = n * [0]
    curr_eps = math.inf
    k = 0

    lower_bounds = []
    upper_bounds = []

    while (curr_eps > eps):
        k += 1
        win_a = vector_addition(win_a, get_column_by_index(C, curr_strategy_b))
        loss_b = vector_addition(loss_b, get_row_by_index(C, curr_strategy_a))
        x[curr_strategy_a] += 1
        y[curr_strategy_b] += 1

        lower_bound = Fraction(min(loss_b), k)
        upper_bound = Fraction(max(win_a), k)
        lower_bounds.append(lower_bound)
        upper_bounds.append(upper_bound)

        curr_eps = min(upper_bounds) - max(lower_bounds)

        curr_strategy_a = get_rand_max_index(win_a)
        curr_strategy_b = get_rand_min_index(loss_b)

    cost = max(lower_bounds) + Fraction(curr_eps, 2)

    x = [Fraction(i, k) for i in x]
    y = [Fraction(i, k) for i in y]

    return x, y, cost

In [42]:
def H(x, y):
    a, b = Fraction(-5, 1), Fraction(9, 2)
    c = Fraction(15, 1)
    d, e = Fraction(-9, 2), Fraction(-9, 1)
  
    return a*x**2 + b*y**2 + c*x*y + d*x + e*y

def find_saddle_point(C):
    max_min = None
    min_max = None

    m = len(C)
    n = len(C[0])

    max_loss = []
    for i in range(n):
        max_loss.append(max(get_column_by_index(C, i)))
    y = get_min_index(max_loss)
    min_max = max_loss[y]

    min_win = []
    for i in range(n):
        min_win.append(min(get_row_by_index(C, i)))
    x = get_max_index(min_win)
    max_min = min_win[x]
    return max_min == min_max, x, y

def average(a):
    return sum(a) / len(a)

def limit(a, eps):
    N = -1
    ff = False
    for i in range(0, len(a) - 1):
        ff = True
        for j in range(i + 1, len(a)):
            if abs(a[j] - a[i]) >= eps:
                ff = False
                break
        if ff:
            N = i
            break
    if not ff:
        return math.inf
    return average([min(a[N + 1: ]), max(a[N + 1: ])])


def calc_grid_element(H, i, j, N):
    return H(Fraction(i, N), Fraction(j, N))

def generate_grid_approximation(H, n):
    cur_H = []
    for i in range(n + 1):
        cur_H.append([0] * (n + 1))
    # fill matrix with calculated values 
    for i in range(n + 1):
        for j in range(n + 1):
            cur_H[i][j] = calc_grid_element(H, i, j, n)
    return cur_H

def grid_approximation_method(H, eps):
    cost_array = []
    x_array = []
    y_array = []
    n = 1
    while(True):
        # create matrix for this iteration
        cur_H = generate_grid_approximation(H, n)
        print("N = {:d}".format(n))
        for i in cur_H:
            print(*["{:8.3f}".format(float(j)) for j in i])
        has_saddle_point, x, y = find_saddle_point(cur_H)
        if has_saddle_point:
            h = cur_H[x][y]
            x = Fraction(x, n)
            y = Fraction(y, n)
            print("Has saddle point: x = {:}, y = {:}, h = {:} = {:.3f}".format(x, y, h, float(h)))
        else:
            print("Has no saddle point")
            x, y, h = brown_robinson_method(cur_H, eps)
            x = Fraction(get_max_index(x), n)
            y = Fraction(get_max_index(y), n)
            print("Calculated with Brown-Robinson method with accuracy eps = {:.3f} solution: x = {:}, y = {:}, h = {:} = {:.3f}".format(float(eps), x, y, h, float(h)))
        cost_array.append(h)
        lim = limit(cost_array, eps)
        if lim != math.inf:
            x_array.append(x)
            y_array.append(y)
        stop_lim = limit(cost_array, Fraction(eps, 10))
        if stop_lim != math.inf:
            return average(x_array), average(y_array), lim
        n += 1

## Main

In [43]:
p = 2
x, y, c = grid_approximation_method(H, Fraction(1, 10**p))
print()
print("Found solution is: x = {:.3f}, y = {:.3f}, h = {:} = {:.3f}".format(float(x), float(y), c, float(c)))

N = 1
   0.000   -4.500
  -9.500    1.000
Has no saddle point
Calculated with Brown-Robinson method with accuracy eps = 0.010 solution: x = 0, y = 1, h = -9003/3160 = -2.849
N = 2
   0.000   -3.375   -4.500
  -3.500   -3.125   -0.500
  -9.500   -5.375    1.000
Has no saddle point
Calculated with Brown-Robinson method with accuracy eps = 0.010 solution: x = 1/2, y = 1/2, h = -77621/24680 = -3.145
N = 3
   0.000   -2.500   -4.000   -4.500
  -2.056   -2.889   -2.722   -1.556
  -5.222   -4.389   -2.556    0.278
  -9.500   -7.000   -3.500    1.000
Has no saddle point
Calculated with Brown-Robinson method with accuracy eps = 0.010 solution: x = 1/3, y = 1/3, h = -4514381/1583820 = -2.850
N = 4
   0.000   -1.969   -3.375   -4.219   -4.500
  -1.438   -2.469   -2.938   -2.844   -2.188
  -3.500   -3.594   -3.125   -2.094   -0.500
  -6.188   -5.344   -3.938   -1.969    0.562
  -9.500   -7.719   -5.375   -2.469    1.000
Has saddle point: x = 1/4, y = 1/2, h = -47/16 = -2.938
N = 5
   0.000   -1.62

Calculated with Brown-Robinson method with accuracy eps = 0.010 solution: x = 1/3, y = 1/2, h = -479267/163584 = -2.930
N = 13
   0.000   -0.666   -1.278   -1.837   -2.343   -2.796   -3.195   -3.541   -3.834   -4.074   -4.260   -4.393   -4.473   -4.500
  -0.376   -0.953   -1.476   -1.947   -2.364   -2.728   -3.038   -3.296   -3.500   -3.651   -3.749   -3.793   -3.784   -3.722
  -0.811   -1.299   -1.734   -2.115   -2.444   -2.719   -2.941   -3.109   -3.225   -3.287   -3.296   -3.251   -3.154   -3.003
  -1.305   -1.704   -2.050   -2.343   -2.583   -2.769   -2.902   -2.982   -3.009   -2.982   -2.902   -2.769   -2.583   -2.343
  -1.858   -2.169   -2.426   -2.630   -2.781   -2.879   -2.923   -2.914   -2.852   -2.737   -2.568   -2.346   -2.071   -1.743
  -2.470   -2.692   -2.861   -2.976   -3.038   -3.047   -3.003   -2.905   -2.754   -2.550   -2.293   -1.982   -1.618   -1.201
  -3.142   -3.275   -3.355   -3.382   -3.355   -3.275   -3.142   -2.956   -2.716   -2.423   -2.077   -1.678   -1.225 

Calculated with Brown-Robinson method with accuracy eps = 0.010 solution: x = 5/17, y = 9/17, h = -999791/342176 = -2.922
N = 18
   0.000   -0.486   -0.944   -1.375   -1.778   -2.153   -2.500   -2.819   -3.111   -3.375   -3.611   -3.819   -4.000   -4.153   -4.278   -4.375   -4.444   -4.486   -4.500
  -0.265   -0.705   -1.117   -1.502   -1.858   -2.187   -2.488   -2.761   -3.006   -3.224   -3.414   -3.576   -3.710   -3.816   -3.895   -3.946   -3.969   -3.965   -3.932
  -0.562   -0.955   -1.321   -1.659   -1.969   -2.252   -2.506   -2.733   -2.932   -3.103   -3.247   -3.363   -3.451   -3.511   -3.543   -3.548   -3.525   -3.474   -3.395
  -0.889   -1.236   -1.556   -1.847   -2.111   -2.347   -2.556   -2.736   -2.889   -3.014   -3.111   -3.181   -3.222   -3.236   -3.222   -3.181   -3.111   -3.014   -2.889
  -1.247   -1.548   -1.821   -2.066   -2.284   -2.474   -2.636   -2.770   -2.877   -2.955   -3.006   -3.029   -3.025   -2.992   -2.932   -2.844   -2.728   -2.585   -2.414
  -1.636   -1.89

Calculated with Brown-Robinson method with accuracy eps = 0.010 solution: x = 2/7, y = 11/21, h = -12337925/4219488 = -2.924

Found solution is: x = 0.292, y = 0.512, h = -18597889913/6358510080 = -2.925
